In [5]:
import sklearn
import pathlib
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
dataset_path = pathlib.Path('./dataset/stocknews/')

In [7]:
data = pd.read_csv(dataset_path / 'Combined_News_DJIA.csv')

In [8]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [9]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

# MODEL 1

In [10]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [11]:
trainheadlines[0]

'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war" b\'BREAKING: Musharraf to be impeached.\' b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\' b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\' b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing" b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side" b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\' b\'Did the U.S. Prep Georgia for War with Russia?\'

In [12]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31675)


In [13]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
predictions = basicmodel.predict(basictest)

In [15]:
pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,61,125
1,92,100


# MODEL 2

In [16]:
advancedvectorizer = CountVectorizer(ngram_range=(3,3))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [17]:
advancedmodel = LogisticRegression(max_iter=1000)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [18]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
advpredictions = advancedmodel.predict(advancedtest)

In [19]:
print(sklearn.metrics.classification_report(test['Label'], predictions))

              precision    recall  f1-score   support

           0       0.40      0.33      0.36       186
           1       0.44      0.52      0.48       192

   micro avg       0.43      0.43      0.43       378
   macro avg       0.42      0.42      0.42       378
weighted avg       0.42      0.43      0.42       378



# Deep

In [ ]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [ ]:
advancedvectorizer = CountVectorizer(ngram_range=(1,1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [ ]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)

In [ ]:
advancedtrain.shape

In [ ]:
advancedtest.shape

# Keras model

In [ ]:
import keras

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Dense(16, activation='sigmoid', input_shape=(advancedtrain.shape[1],)))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(4, activation='relu'))

model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    advancedtrain,
    train['Label'],
    batch_size=100,
    epochs=15,
    validation_data=(advancedtest, test['Label'])
)

# Binary classifier model

In [7]:
import spacy

In [8]:
def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [9]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [10]:
trainheadlines = []
labelstrain = []
for row in range(0,len(train.index)):
    trainheadlines.extend(str(x) for x in train.iloc[row,2:27])
    labelstrain.extend( [train.iloc[row, 1],]* 25 )

In [11]:
testheadlines = []
labelstest = []
for row in range(0,len(test.index)):
    testheadlines.extend(str(x) for x in test.iloc[row,2:27])
    labelstest.extend( [test.iloc[row, 1],] * 25 )

In [21]:
import re

brief_cleaning = (re.sub("[^A-Za-z\"]+", ' ', str(row)).lower() for row in trainheadlines)

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

In [22]:
txt[:10]

['b"georgia down russian warplane country brink war "',
 'b break musharraf impeach',
 'b russia today column troop roll south ossetia footage fight youtube',
 'b russian tank move capital south ossetia reportedly completely destroy georgian artillery fire',
 'b"afghan child rape impunity u n official say sick year old rape "',
 'b russian tank enter south ossetia whilst georgia shoot russian jet',
 'b"break georgia invade south ossetia russia warn intervene s "',
 'b"the enemy combatent trial sham salim haman sentence year keep longer feel like "',
 'b georgian troop retreat s osettain capital presumably leave people kill video',
 'b u s prep georgia war russia']

In [ ]:
# advancedvectorizer = CountVectorizer(ngram_range=(2,2))
advancedvectorizer = CountVectorizer()
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
advancedtest = advancedvectorizer.transform(testheadlines)

In [ ]:
labelstrain = np.array(labelstrain)
labelstest = np.array(labelstest)

In [ ]:
print(advancedtrain.shape)
print(advancedtest.shape)
print(labelstrain.shape)
print(labelstest.shape)

In [ ]:
import keras

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Dense(16, activation='sigmoid', input_shape=(advancedtrain.shape[1],)))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.9))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.9))

model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    advancedtrain,
    labelstrain,
    batch_size=100,
    epochs=5,
    validation_data=(advancedtest, labelstest)
)